<a href="https://colab.research.google.com/github/Grystniu/U-net-restoration/blob/main/U_NET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install unrar

In [ ]:
!unzip /content/result_x.zip

In [ ]:
!unzip /content/result_y.zip

In [ ]:
!unzip /content/result_y_new.zip

In [ ]:
!unzip /content/result_x_new.zip

In [ ]:
import sys
sys.path.append('/content/modules.py')
from modules import *

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

def U_Net(input_size, input_channels, filters, learning_rate):
    X = Input(shape=(input_size, input_size, input_channels))

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(X)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up4 = Concatenate()([UpSampling2D(size=(2, 2))(conv3), conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = Concatenate()([UpSampling2D(size=(2, 2))(conv4), conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)

    model = Model(inputs=X, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [ ]:
path2 = '/content/result_x'
path = '/content/result_y'
train_set_X, train_set_Y, val_set_X, val_set_Y, test_set_X, test_set_Y = read_data(path, path2)

In [ ]:
unet = load_unet()

In [ ]:
unet = U_Net(input_size=128, input_channels=1, filters=64, learning_rate=0.001)
history = unet.fit(x=train_set_X, y=train_set_Y, validation_data=(val_set_X, val_set_Y), batch_size=32, epochs=15)

In [ ]:
import tensorflow as tf
y_pred = unet.predict(test_set_X)
print('average SSIM of U-Net on the testing set: {}'.format(tf.math.reduce_mean(tf.image.ssim(test_set_Y, y_pred, 1.)).numpy()))
# Выводим несколько результатов выходных данных U-Net с обучающей выборки
print_model_outputs(unet, train_set_X[:4], train_set_Y[:4], 'Training Data')

# Выводим несколько результатов выходных данных U-Net с валидационной выборки
print_model_outputs(unet, val_set_X[:4], val_set_Y[:4], 'Validation Data')

# Выводим несколько результатов выходных данных U-Net с тестовой выборки
print_model_outputs(unet, test_set_X[:4], test_set_Y[:4], 'Testing Data')

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
pred_new = read_new_images('/content/result_x_new')
pred_origin = read_new_images('/content/result_y_new')
My_predictions = unet.predict(pred_new)

In [ ]:
print('average SSIM of U-Net on the testing set: {}'.format(tf.math.reduce_mean(tf.image.ssim(pred_origin, My_predictions, 1.)).numpy()))

In [ ]:
# Выводим несколько результатов выходных данных U-Net с тестовой выборки
print_model_outputs(unet, pred_new, pred_origin, 'New_prediction')

In [ ]:
save_unet(unet)

In [ ]:
source_folder = '/content/wl'
destination_folder = '/content/wlresult'
model = load
processed_images = read_single_image_and_save(source_folder, destination_folder, model=model)
import os
import shutil

source_folder = '/content/wlresult'

temp_folder = '/content/temp_archive'
os.makedirs(temp_folder, exist_ok=True)

for file_name in os.listdir(source_folder):
    file_path = os.path.join(source_folder, file_name)
    if os.path.isfile(file_path):
        shutil.copy(file_path, temp_folder)

shutil.make_archive('/content/temp_archive', 'zip', temp_folder)

print("Архив создан.")

shutil.move('/content/temp_archive.zip','/content/temp_archive.zip' )

shutil.rmtree(temp_folder)